In [3]:
!mkdir weights
!wget https://github.com/nttcslab/m2d/releases/download/v0.1.0/msm_mae_vit_base-80x608p16x16-220924-mr75.zip \
  -O weights/msm_mae_vit_base-80x608p16x16-220924-mr75.zip

mkdir: cannot create directory ‘weights’: File exists


In [7]:
%cd weights
!unzip msm_mae_vit_base-80x608p16x16-220924-mr75.zip

[Errno 2] No such file or directory: 'weights'
/home/alc/TIL-2023/SpeakerID/m2d/weights
Archive:  msm_mae_vit_base-80x608p16x16-220924-mr75.zip
   creating: msm_mae_vit_base-80x608p16x16-220924-mr75/
  inflating: msm_mae_vit_base-80x608p16x16-220924-mr75/checkpoint-300.pth  


In [2]:
eval('evar.ar_m2d.AR_M2D')

NameError: name 'evar' is not defined

In [3]:
%cd ~/Projects/TIL-2023/SpeakerID/m2d
import sys
sys.path.append("./evar")

import torch
from pathlib import Path
from evar.ar_m2d import AR_M2D
from finetune import TaskNetwork
from lineareval import make_cfg
from evar.common import kwarg_cfg

device = torch.device("cuda")

cfg, n_folds, activation, balanced = make_cfg(
  config_file="evar/config/m2d.yaml",
  task="til",
  options="",
)
cfg.weight_file = "/home/alc/TIL-2023/SpeakerID/m2d/weights/msm_mae_vit_base-80x608p16x16-220924-mr75/checkpoint-300.pth"
cfg.unit_sec = 15.0
# change n_class according to no. of unique speakers
cfg.runtime_cfg = kwarg_cfg(n_class=2, hidden=())

run_path = Path("/home/alc/TIL-2023/SpeakerID/m2d/evar/logs/til_ar_m2d.AR_M2D_64f5263c")
state_dict = torch.load(run_path / "weights_ep0it0-0.66667_loss0.3365.pth")
norm_stats = torch.load(run_path / "norm_stats.pt")

backbone = AR_M2D(cfg, inference_mode=True, norm_stats=norm_stats).to(device)
model = TaskNetwork(cfg, ar=backbone).to(device)
model = torch.nn.DataParallel(model).to(device)

model.load_state_dict(state_dict)
model.eval()

/home/yip/Projects/TIL-2023/SpeakerID/m2d


/home/yip/.local/lib/python3.10/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(
ERROR:root:Install toprchopenl3.
>>> pip install torchopenl3
ERROR:root:Make your copy of VGGish under external folder. Check Preparing-models.md for the details.
2023-06-10 16:50:56.510442: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 16:51:05.333446: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ERROR:root:Install tensorflow and tensorflow_hub.
>>> pip install tensorflow tensorflow_hub
ERROR:root:Make your copy of COALA under external folder. 


+task_metadata=evar/metadata/til.csv,+task_data=work/16k/til,+unit_samples=240000


FileNotFoundError: [Errno 2] No such file or directory: '/home/alc/TIL-2023/SpeakerID/m2d/evar/logs/til_ar_m2d.AR_M2D_64f5263c/weights_ep0it0-0.66667_loss0.3365.pth'

In [14]:
import torchaudio
import pickle
import torch

audio = torchaudio.load("/home/alc/TIL-2023/SpeakerID/m2d/evar/work/16k/til/valid-2.wav")[0]
classes_pickle_path = "/home/alc/TIL-2023/SpeakerID/m2d/evar/logs/til_ar_m2d.AR_M2D_64f5263c/classes.pkl"

with open(classes_pickle_path, "rb") as classes_pickle_file:
  classes = pickle.load(classes_pickle_file)

classes = list(classes)

model_output = model(audio)
pred_idx = torch.argmax(model_output[0])
pred_class_name = classes[pred_idx]

pred_class_name

'1'